# Depuracion e integracion de datos mixtos

El presente notebook realiza la exploración inicial y depuración de datos agregados de vegetación consolidados por el SMBYC.
Se requieren los siguientes archivos:

1. `asignacion`: Tabla en formato csv. Contiene información básica de las parcelas (ubicación, custodio, etc.).
2. `individuos`: Tabla en formato csv. Contiene medidas y demás datos relacionos con los individuos (diámetro, altura, parcela, placa, etc.). 

In [1]:
import pandas as pd
import numpy as np
%matplotlib inline

# Asignar nombres de archivos a variables
asignacion = "../data/quimera/asignacion.csv"
individuos =  "../data/quimera/newind.csv"

In [154]:
asig = pd.read_csv(asignacion)
ind = pd.read_csv(individuos, low_memory=False) # low_memory asume que los datos estan homogeneamente tipificados.
#tot = ind.join(asig.set_index('Plot'), on='Plot', rsuffix='_a') # concatenacion de ambas tablas usando en indice de parcela (`Plot`) como columna de referencia

### Variables para los nombres de las columnas

A continuación se adjudica una variable a cada nombre de la columna en ambos archivos. Si los nombres no son los mencionados deben ser actualizados en concordancia. El nombre de la columna que contiene el indice de la parcela debe ser igual en ambos archivos.

In [3]:
# Variables de campos de tabla `individuos`

FID = 'FID' # Indice de medicion (int)
Plot = 'Plot' # Indice parcela (int)
D = 'D'# Diametro del tallo en cm (float)
H = 'H' # Altura total del tallo en m (float)
X = 'X' # Coordenada X en m (float)
Y = 'Y' # Coordenada Y en m (float)
Placa = 'Placa' # Placa de referencia del individuo (str, aunque la mayoria son int)
Densidad = 'Densidad' # Densidad de la madera en gr/ml (float)
Fuente_densidad = 'Fuente_densidad' # Referencia bibliografica de la densidad de la madera (str)
Habito = 'Habito' # Clasificacion de referencia (str: 'Arborea', 'Palma', 'No-Arborea', 'Exotica', 'Paramo', 'Mangle').
Entra_calculos = 'Entra_calculos' # Inclusion en analisis (str: 'Si', 'No')
Familia = 'Familia' # Familia taxonomica (str)
Autor_familia = 'Autor_familia' # Autor familia taxonomica (str)
Genero = 'Genero' # Genero taxonomica (str)
Autor_genero = 'Autor_genero' # Autor genero taxonomica (str)
Estado_epiteto = 'Estado_epiteto' # Incertidumbre de la determinacion especifica (str: 'aff.', 'cf.', 'vs.', 'gr.')
Epiteto = 'Epiteto' # Epiteto especifico, si indeterminado a especie contiene 'sp.' (str)
Autor_especie = 'Autor_especie' # Autor epiteto taxonomico (str)
Morfoespecie = 'Morfoespecie' # Concatenacion de los campos Genero y Epiteto (str).

fields_ind = [FID, Plot, D, H, X, Y, Placa, Densidad, Fuente_densidad, Habito, Entra_calculos, Familia, Autor_familia, Genero, Autor_genero, Estado_epiteto, Epiteto, Autor_especie, Morfoespecie]


In [4]:
# Variables de campos de tabla `asignacion`

Area = 'Area' # Superficie de la parcela en hectareas (float64)
Year = 'Year' # Año de levantamiento de datos (int64)
Tipo_parcela = 'Tipo_parcela' # Clase de parcela (str: 'Temporal', 'Permanente')
Custodio = 'Custodio' # Autor o custodio de la informacion (str)
Custodio_abreviado = 'Custodio_abreviado' # Abreviatura del autor o custodio de la informacion (str)
Parcela_original = 'Parcela_original' # Codigo de la parcela empleado por el custodio (str)
Proyecto = 'Proyecto' # Codigo del proyecto bajo el cual la parcela fue establecida (str)
PID = 'PID' # Que carajos es esto???????????????
X = 'X' # Coordenada X WGS84 zona 18N en m, origen en 0°, -75° (float64). Reformatear a numerico, aunque hay unos valores con dobles comas.
Y = 'Y' # Coordenada x WGS84 zona 18N en m, origen en 0°, -75° (float64). Reformatear a numerico, aunque hay unos valores con dobles comas.
X_MAGNA = 'X_MAGNA' # Coordenada x Magna en m (float64). Reformatear a numerico, aunque hay unos valores con dobles comas.
Y_MAGNA = 'Y_MAGNA' # Coordenada y Magna en m (float64). Reformatear a numerico, aunque hay unos valores con dobles comas.
Acceso = 'Acceso' # Clase de acceso permitido al IDEAM por el custodio (str: 'Confidencial','Público').
Departamento = 'Departamento' # Departamento str.
Municipio = 'Municipio' # Municipio str.
CAR = 'CAR' # Corporacion autonoma regional con jurisdiccion en el sitio de muestreo (str).
UAESPNN = 'UAESPNN' # Unidad del sistema de areas protegidas con jurisdiccion en el sitio de muestreo (str).
Region = 'Region' # Region geografica de Colombia (str: 'Amazonia', 'Caribe', 'Andes', 'Pacifico', 'Orinoquia', 'Andina').
Escenario_referencia = 'Escenario_referencia' # Unidad geografica de referencia (str: 'Amazonia', 'Noroccidental', 'Caribe', 'Suroccidental', 'Antioquia','Andes oriental', 'Eje cafetero', 'Nororiental', 'Orinoquia')
ECP = 'ECP' # ????????????????????????????????? (float64)
Holdridge = 'Holdridge' # Clasificacion climatica de Holdridge, modelo 2014 (str)
Provincia = 'Provincia' # Provincia bioclimatica, modelo 2014 (str: 'Wet forest', 'Moist forest', 'Dry forest')
Caldas_Lang = 'Caldas_Lang' # Clasificacion climatica Caldas-Lang, modelo 2014 (str)
Martonne = 'Martonne' # Clasificacion climatica Martonne, modelo 2014 (str: 'Bosque lluvioso','Bosque lluvioso estacional','Bosque húmedo','Bosque subhúmedo')
Eq1 = 'Eq1' # Valor de biomasa al aplicar la ecuacion 1 (int64)??????
Eq2 = 'Eq2' # Valor de biomasa al aplicar la ecuacion 2 (int64)??????
Eq3 = 'Eq3' # Valor de biomasa al aplicar la ecuacion 3 (int64)??????

fields_asig = [Plot, Area, Year, Tipo_parcela, Custodio, Custodio_abreviado, Parcela_original, Proyecto, PID, X, Y, X_MAGNA, Y_MAGNA, Acceso, Departamento, Municipio, CAR, UAESPNN, Region, Escenario_referencia, ECP, Holdridge, Provincia, Caldas_Lang, Martonne, Eq1, Eq2, Eq3]

### Verificar si las columnas tienen el tipo de dato apropiado

In [155]:
# Verificar si todas las columnas tienen el tipo de dato adecuado

for fi in [FID, Plot]:
    if ind[fi].dtype != np.int64:
        print "Campo {0} tiene tipo inapropiado ({1} en vez de int64).".format(fi, ind[fi].dtype)
        
for fi in [D, H, X, Y, Densidad]:
    if ind[fi].dtype != np.float64:
        print "Campo {0} tiene tipo inapropiado ({1}en vez de float64).".format(fi, ind[fi].dtype)
        
for fi in [Placa, Fuente_densidad, Habito, Familia, Autor_familia, Genero, Autor_genero, Estado_epiteto, Epiteto, Autor_especie, Morfoespecie]:
    non_strings = ind[fi].dropna()[~ind[fi].dropna().apply(type).eq(str)]
    if len(non_strings):
        print "Campo {0} tiene tipo inapropiado ({1} en vez de str).".format(fi, non_strings.dtype)

try:
    ind[Entra_calculos].replace(to_replace = ['Si','No'], value = [True, False], inplace = True)
except TypeError, ErrorMessage:
    if ErrorMessage.args[0] == "Cannot compare types 'ndarray(dtype=bool)' and 'str'":
        pass
    else:
        raise
except:
    raise

if ind[Entra_calculos].dtype != np.bool_:
    print "Campo {0} tiene tipo inapropiado ({1} en vez de str).".format(Entra_calculos, ind[Entra_calculos].dtype)


# Verificacion de tipo de datos en tabla asignacion    

# Campo `Acceso` es insertado en la columna `AccesoPublico` de la base de datos, que es boolean. Sin embargo, para poder importarlo via mysqlimport es guardado en pandas como np.int64

asig[Acceso].replace(to_replace = ['Confidencial','Público'], value = [0, 1], inplace = True)
asig[X].replace(to_replace=r',',value='',regex=True, inplace=True)
asig[Y].replace(to_replace=r',',value='',regex=True, inplace=True)
asig[X_MAGNA].replace(to_replace=r',',value='',regex=True, inplace=True)
asig[Y_MAGNA].replace(to_replace=r',',value='',regex=True, inplace=True)
asig[[X,Y,X_MAGNA,Y_MAGNA]] = asig[[X,Y,X_MAGNA,Y_MAGNA]].apply(pd.to_numeric)
for fi in [Plot, Year, X, Y, X_MAGNA, Y_MAGNA, Eq1, Eq2, Eq3, Acceso]:
    if asig[fi].dtype != np.int64:
        print "Campo {0} tiene tipo inapropiado ({1} en vez de int64).".format(fi, asig[fi].dtype)
        
for fi in [Area]:
    if asig[fi].dtype != np.float64:
        print "Campo {0} tiene tipo inapropiado ({1} en vez de float64).".format(fi, asig[fi].dtype)

for fi in [Tipo_parcela, Custodio, Custodio_abreviado, Parcela_original, Departamento, Municipio, CAR, UAESPNN, Region, Escenario_referencia, Holdridge, Provincia, Caldas_Lang, Martonne]:
    non_strings = asig[fi].dropna()[~asig[fi].dropna().apply(type).eq(str)]
    if len(non_strings):
        print "Campo {0} tiene tipo inapropiado ({1} en vez de str).".format(fi, non_strings.dtype)
        
    


### Verificar si los valores estan en el rango apropiado

En este punto del proceso sólo se verifica que los valores tengan sentido lógico. Una mayor depuración de los datos se realizará después de que éstos sean incluidos en la base de datos.

In [156]:
#########################################
# Tabla individuos
#########################################

# Indice no debe contener duplicado
if len(ind[ind[FID].duplicated()]):
    print "Tabla {0} contiene indices duplicados.".format(individuos)
    
# Rango de diametro = 10-770 (diametro de General Sherman)
if ind[D].min() < 10:
    print "Existen valores de diametro inferiores a 10 cm"
if ind[D].max() > 770:
    print "Existen valores de diametro dudosamente altos"
    
# Rango de alturas = 1 - 100
if ind[H].min() < 1:
    print "Existen valores de altura inferiores a 1 m"
if ind[H].max() > 770:
    print "Existen valores de altura dudosamente altos"

# Rango de densidades de madera = 0.08 - 1.39 (Global Wood Density Database compilada por Chave y diponible en http://datadryad.org/repo/handle/10255/dryad.235)
if ind[Densidad].min() < 0.08:
    print "Existen valores de densidad inferiores a 0.8 gr/ml"
if ind[Densidad].max() > 1.39:
    print "Existen valores de densidad superiores a 1.39 gr/ml"
    
# Estados de incertidumbre taxonomica = 'aff.', 'cf.', 'vel sp. aff.'
########################################################
# Reemplazo de valores no aceptados detectados:
# 'vs.' reemplazado con 'vel sp. aff.'
# 'gr.' reemplazado con 'vel sp. aff.'
########################################################
ind[Estado_epiteto].replace(to_replace = ['vs.', 'gr.'], value = ['vel sp. aff.','vel sp. aff.'], inplace = True)

for est in ind[Estado_epiteto].dropna().unique():
    if est not in ['aff.', 'cf.', 'vel sp. aff.']:
        print "{0} no es un estado de incertidumbre de determinacion aceptado".format(est)
        print ind[[Genero, Estado_epiteto, Epiteto]][ind[Estado_epiteto] == est]

# Informacion de la columna `Habito` debe ser distribuido en tres columnas: `Habito`,
# `Origen` y `Ecosistema`. Columnas `Origen` y `Ecosistema` deben ser creadas para tal 
# proposito

ind.insert(10, 'Origen', value='Nativa')
ind.insert(12, 'Ecosistema', value=np.nan)
ind.loc[ind[Habito] == 'No-Arborea', Habito] = 'No arborea'
ind.loc[ind[Habito] == 'Exotica', 'Origen'] = 'Introducida'
ind.loc[ind[Habito] == 'Paramo' , 'Ecosistema'] = 'Paramo'
ind.loc[ind[Habito] == 'Mangle' , 'Ecosistema'] = 'Manglar'
ind[Habito].replace(to_replace = ['Exotica','Paramo','Mangle'], value = np.nan, inplace = True)

# Verificar que `Habito`, `Origen` y `Ecosistema` contienen valores validos
for hab in ind[Habito].dropna().unique():
    if hab not in ['Arborea', 'Palma', 'Liana', 'No arborea']:
        print "{0} no es un valor aceptado de `Habito`".format(hab)
        print ind[[Genero, Epiteto, Habito]][ind[Habito] == hab]
        
for ori in ind['Origen'].dropna().unique():
    if ori not in ['Nativa', 'Introducida']:
        print "{0} no es un valor aceptado de `Origen`".format(ori)
        print ind[[Genero, Epiteto, 'Origen']][ind['Origen'] == ori]

for eco in ind['Ecosistema'].dropna().unique():
    if eco not in ['Paramo', 'Manglar']:
        print "{0} no es un valor aceptado de `Ecosistema`".format(eco)
        print ind[[Genero, Epiteto,'Ecosistema']][ind['Ecosistema'] == eco]


#########################################
# Tabla asignacion
#########################################

# Indice de parcela no debe tener duplicados
if len(asig[asig[Plot].duplicated()]):
    print "Tabla {0} contiene indices duplicados.".format(asignacion)
    
# Verificar areas, rango permitido: 0.02-25 ha
if asig[Area].min() < 0.02:
    print "Algunos valores de area de parcela estan por debajo del valor minimo"
if asig[Area].max() > 25:
    print "Algunos valores de area de parcela son sospechosamente altos"
    
# Rango de años permitido: 1990-2017
if asig[Year].min() < 1990:
    print "Realmente hay datos levantados antes de 1990?"
if asig[Year].max() > 2017:
    print "Parece que algunos datos provienen del futuro. Verificar fechas."

# Verificar correspondencia 1:1 entre el indice propio de parcela y el indice del custodio  
if filter(lambda x: x != 1, asig.groupby([Plot, Parcela_original]).size()):
    print "No hay concordancia entre las asignaciones de indices propios y externos:"
    multiPlots = asig.groupby([Plot, Parcela_original]).size().reset_index()
    print multiPlots[multiPlots[0] > 1]
    
# Verificar tipo de parcela valido: 'temporal', 'permanente'
for par in asig[Tipo_parcela].dropna().unique():
    if par not in ['Permanente','Temporal']:
        print "{0} no es un tipo de parcela aceptado".format(par)

# Varificar rango de coordenadas.
x_rango = (-231000, 1410000)
y_rango = (-480000, 1500000)
if asig[X].min() < x_rango[0] or asig[X].max() > x_rango[1]:
    print "Coordenadas X fuera del rango permitido"
if asig[Y].min() < y_rango[0] or asig[Y].max() > y_rango[1]:
    print "Coordenadas Y fuera del rango permitido"

# Verificar rango Magna
xm_rango = (167000, 1810000)
ym_rango = (23000, 1900000)
if asig[X_MAGNA].min() < xm_rango[0] or asig[X_MAGNA].max() > xm_rango[1]:
    print "Coordenadas X Magna fuera del rango permitido"
if asig[Y_MAGNA].min() < ym_rango[0] or asig[Y_MAGNA].max() > ym_rango[1]:
    print "Coordenadas Y Magna fuera del rango permitido"

for acc in asig[Acceso].unique():
    if acc not in [0, 1]:
        print "Clase de acceso a datos de parcela tiene valores no aceptados"

# Creacion de archivos csv de inicializacion de la base de datos en MySQL

La generacion de la base de datos en MySQL es realizada eficientemente a traves del comando `mysqlimport`.
Con dicho ejecutable se pueden importar tablas contenidas en archivos csv en tiempos de ejecucion muy cortos.
De esta manera es necesaria la creacion de tablas en archivos csv de manera temporal, que contenga los datos una vez depurados.
Adicionalmente, el nombre de las columnas y su orden debe concordar con aquellos del las tabls de la base de datos MySQL.
El nombre del archivo csv tambien debe concordar con el nombre de la tabla a poblar en la base de datos.

## Tabla Parcelas

Despues de crear el archivo __Parcelas.csv__ la tabla es importado a la base de datos con el comando `mysqlimport --ignore-lines=1 --fields-terminated-by=,  --local --columns='PlotID,Area,Custodio,CustodioAbbreviado,Proyecto,X,Y,XMAGNA,YMAGNA, Departamento,Municipio,CAR,UAESPNN,Region,EscenarioReferencia,AccesoPublico' -u root -p  Quimera ../data/mysql_csv/Quimera/Parcelas.csv`

Un problema aun no resuleto es que mysqlimport no genera valores nulos automaticamente.

In [59]:
asig[[Plot, Area, Custodio, Custodio_abreviado, Proyecto, X, Y, X_MAGNA, Y_MAGNA, Departamento, Municipio, CAR, UAESPNN, Region, Escenario_referencia, Acceso]].to_csv('../data/mysql_csv/Quimera/Parcelas.csv', index=False)


## Tabla Taxonomia

Despues de crear el archivo __Taxonomia.csv__ la tabla es importado a la base de datos con el comando `mysqlimport --ignore-lines=1 --fields-terminated-by=,  --local --columns='TaxonID,Familia,Genero,AutorGenero,Epiteto,AutorEpiteto,Habito,Origen' -u root -p  Quimera ../data/mysql_csv/Quimera/Taxonomia.csv`

Un problema aun no resuleto es que mysqlimport no genera valores nulos automaticamente.

In [177]:
ind = pd.read_csv(individuos, low_memory=False)
ind2 = ind.fillna(value = {Familia:'NAN',Genero:'NAN',Epiteto:'NAN'})
tax = ind2.groupby(by=[Familia, Genero, Autor_genero, Epiteto, Autor_especie, Habito]).size().reset_index()
tax[tax.Genero == 'NAN']

,Familia,Genero,Autor_genero,Epiteto,Autor_especie,Habito,0


In [157]:
tax = ind.groupby(by=[Familia, Genero, Autor_genero, Epiteto, Autor_especie, Habito, 'Origen']).size().reset_index().drop(columns = 0)

tax.index += 1

tax.to_csv('../data/mysql_csv/Quimera/Taxonomia.csv', index_label = 'TaxonID')

In [166]:
ind.Genero[ind.Genero == 'NAN']


0         NAN
6         NAN
7         NAN
9         NAN
19        NAN
27        NAN
32        NAN
38        NAN
60        NAN
61        NAN
66        NAN
67        NAN
77        NAN
81        NAN
86        NAN
91        NAN
96        NAN
99        NAN
103       NAN
116       NAN
120       NAN
121       NAN
137       NAN
141       NAN
146       NAN
203       NAN
215       NAN
218       NAN
219       NAN
220       NAN
         ... 
583275    NAN
583296    NAN
583315    NAN
583319    NAN
583335    NAN
583338    NAN
583339    NAN
583350    NAN
583360    NAN
583364    NAN
583377    NAN
583380    NAN
583381    NAN
583396    NAN
583400    NAN
583435    NAN
583451    NAN
583458    NAN
583464    NAN
583508    NAN
583540    NAN
583542    NAN
583546    NAN
583552    NAN
583557    NAN
583563    NAN
583566    NAN
583567    NAN
583576    NAN
583579    NAN
Name: Genero, Length: 72440, dtype: object

In [159]:
tax.Genero[tax.Genero == 'NAN']

Series([], Name: Genero, dtype: object)

In [143]:
dets = ind[[FID, Familia, Genero, Epiteto, Estado_epiteto]].merge( tax.reset_index( ).rename(columns = {'index':'TaxonID'} ), how = 'left', on=[Familia, Genero, Epiteto])

dets.index += 1

dets
#dets[['TaxonID', Estado_epiteto]].to_csv('../data/mysql_csv/Quimera/Determinaciones.csv', index_label = 'DetID')

,FID,Familia,Genero,Epiteto,Estado_epiteto,TaxonID,Autor_genero,Autor_especie,Habito,Origen
1,1,Chrysobalanaceae,NAN,jbr.1505,NaN,NaN,NaN,NaN,NaN,NaN
2,2,Lecythidaceae,Eschweilera,punctata,NaN,2156.0,Mart. ex DC.,S.A. Mori,Arborea,Nativa
3,3,Lecythidaceae,Eschweilera,punctata,NaN,2156.0,Mart. ex DC.,S.A. Mori,Arborea,Nativa
4,4,Fabaceae,Machaerium,jbr.1507,NaN,NaN,NaN,NaN,NaN,NaN
5,5,Sapotaceae,Micropholis,guyanensis,NaN,3709.0,(Griseb.) Pierre,(A. DC.) Pierre,Arborea,Nativa
6,6,Lauraceae,Mezilaurus,itauba,NaN,2000.0,Kuntze ex Taub.,(Meisn.) Taub. ex Mez,Arborea,Nativa
7,7,Clusiaceae,NAN,jbr.1510,NaN,NaN,NaN,NaN,NaN,NaN
8,8,Lecythidaceae,NAN,jbr.1511,NaN,NaN,NaN,NaN,NaN,NaN
9,9,Arecaceae,Oenocarpus,bataua,NaN,442.0,Mart.,Mart.,Palma,Nativa
10,10,Myrtaceae,NAN,jbr.1513,NaN,NaN,NaN,NaN,NaN,NaN


In [121]:
ind[[Genero, Epiteto]].head(30).fillna('')

,Genero,Epiteto
0,,jbr.1505
1,Eschweilera,punctata
2,Eschweilera,punctata
3,Machaerium,jbr.1507
4,Micropholis,guyanensis
5,Mezilaurus,itauba
6,,jbr.1510
7,,jbr.1511
8,Oenocarpus,bataua
9,,jbr.1513
